In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import optim, nn
import torch.nn.functional as F
from numpy import genfromtxt
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks

Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/My Drive/Colab Notebooks'
/content


In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir('drive/4242')
!ls

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
train_features = pd.read_csv("./dengue_features_train.csv")
train_labels = pd.read_csv("./dengue_labels_train.csv")

# train_features = pd.read_csv("./content/drive/Shared drives/Colab\ Notebooks")

FileNotFoundError: ignored

In [ ]:
sj_train_features=train_features[train_features.city=='sj']

In [ ]:
sj_train_features.drop('city',axis=1, inplace=True)
#sj_train_features.drop('week_start_date',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
sj_train_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 936 entries, 0 to 935
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   year                                   936 non-null    int64  
 1   weekofyear                             936 non-null    int64  
 2   week_start_date                        936 non-null    object 
 3   ndvi_ne                                745 non-null    float64
 4   ndvi_nw                                887 non-null    float64
 5   ndvi_se                                917 non-null    float64
 6   ndvi_sw                                917 non-null    float64
 7   precipitation_amt_mm                   927 non-null    float64
 8   reanalysis_air_temp_k                  930 non-null    float64
 9   reanalysis_avg_temp_k                  930 non-null    float64
 10  reanalysis_dew_point_temp_k            930 non-null    float64
 11  reanal

In [ ]:
sj_train_labels=train_labels[train_labels.city=='sj']
sj_train_labels.drop('city',axis=1, inplace=True)

In [ ]:
train_features = pd.read_csv("./dengue_features_train.csv")
train_labels = pd.read_csv("./dengue_labels_train.csv")

sj_train_features=train_features[train_features.city=='sj']
sj_train_features.drop(['city','week_start_date'],axis=1, inplace=True)

sj_train_labels=train_labels[train_labels.city=='sj']
sj_train_labels.drop('city',axis=1, inplace=True)

In [ ]:
#After preprocessing,transform to tensor
sj_train_features_t=torch.tensor(sj_train_features.values,dtype=torch.float)
sj_train_labels_t=torch.tensor(sj_train_labels.total_cases.values,dtype=torch.float).view(-1,1)

Loss function. log rsme is used as as rsme is too large.


In [ ]:
loss=nn.MSELoss()

def log_rmse(net,features,labels):
    with torch.no_grad():
        #rmse always too high,and the scale is large, so use log 
        clipped_preds=torch.max(net(features),torch.tensor(1.0))
        rmse=torch.sqrt(2*loss(clipped_preds.float().log(),labels.float().log()).mean())
    return rmse.item() 

Initialise neural network linearly.

In [ ]:
#initialize net,linear~
def get_net(feature_num):
    net=nn.Linear(feature_num,1)
    
    #parameters() generate tensor data：
    for param in net.parameters():
        nn.init.normal_(param,mean=0,std=0.01)
    
    return net

Training codes.

 ADAM optimizer is used for it's stochastic gradient descent method to update biases

Backpropagation is also used.


In [ ]:
#Train model
def train(net,train_features,train_labels,test_features,test_labels,
          num_epochs,learning_rate,weight_decay,batch_size):
    #Loss
    train_ls=[] 
    dataset=utils.data.TensorDataset(train_features,train_labels)
    train_iter=utils.data.DataLoader(dataset,batch_size,shuffle=True)
    #optimizer：Adam
    optimizer=optim.Adam(params=net.parameters(),lr=learning_rate,weight_decay=weight_decay)
    net=net.float()
    for epoch in range(num_epochs):
        for x , y in train_iter:
            #Forward：
            output=net(x.float())
            #Loss：
            l=loss(output,y.float())
            #zero
            optimizer.zero_grad()
            #backward
            l.backward()
            #optimizer
            optimizer.step()
        train_ls.append(log_rmse(net,train_features,train_labels))  #train
    return train_ls